In [88]:
import os
import random
import subprocess
from collections import defaultdict

from mido import Message, MidiFile, MidiTrack, MetaMessage
from pydub import AudioSegment

# === CONFIG ===
BPM = 200
BEATS = 12
BASE_MIDI_NOTE = 61

soundfont_path = os.path.expanduser("~/soundfonts/FluidR3_GM.sf2")
taal_path = os.path.expanduser("~/Documents/music/ektaal_200bpm_csharp.wav")
taan_midi_path = "taan.mid"
taan_wav_path = "taan.wav"
mixed_path = "mixed.wav"

# raga_notes = [0, 2, 4, 6, 7, 9, 11, 12]
# low octave: Pa (-5) to Sa (0)
# middle octave: Sa (0) to Sa' (12)
# high notes up to Ga' (16)

SWARA_DEGREES = {
    "S": 0, "R": 2, "G": 4, "M": 6, "P": 7, "D": 9, "N": 11,
}

def swara_to_note(swara: str) -> int:
    up = swara.count("'")
    down = swara.count("_")
    base = swara.strip("'_")
    if base not in SWARA_DEGREES:
        raise ValueError(f"Invalid swara: {swara}")
    return SWARA_DEGREES[base] + 12 * (up - down)

def note_to_swara(note: int) -> str:
    octave_offset, base_pitch = divmod(note, 12)
    inv = {v: k for k, v in SWARA_DEGREES.items()}
    if base_pitch not in inv:
        raise ValueError(f"Note {note} not mapped to swara")

    base = inv[base_pitch]
    if octave_offset == 0:
        return base
    elif octave_offset > 0:
        return base + ("'" * octave_offset)
    else:
        return ("_" * abs(octave_offset)) + base

notes = [
    -5, -3, -1, 0, 2, 4, 6, 7, 9, 11, 12, 14, 16
]
note_names = {n: note_to_swara(n) for n in notes}



# === TAAN GENERATOR ===
class TaanGenerator:
    def __init__(self, notes):
        self.notes = notes
        self.transitions = defaultdict(self._init_uniform)

    def _init_uniform(self):
        prob = 1.0 / len(self.notes)
        return defaultdict(lambda: prob, {n: prob for n in self.notes})

    def observe(self, a, b, c, w=1.0):
        self.transitions[(a, b)][c] += w

    def normalize(self):
        self.decay_all()
        for key, targets in self.transitions.items():
            total = sum(targets.values())
            if total == 0:
                continue  # skip dead branches
            for c in targets:
                targets[c] /= total
        


    def sample_next(self, a, b):
        choices = self.transitions.get((a, b))
        if not choices:
            return random.choice(self.notes)
        return random.choices(list(choices), weights=list(choices.values()))[0]

    def generate(self, beats, start_note=0, end_note=0, tries=1000):
        length = beats * 2
        for _ in range(tries):
            taan = [start_note, random.choice([n for n in self.notes if n != start_note])]
            while len(taan) < length:
                taan.append(self.sample_next(taan[-2], taan[-1]))
            if taan[-1] == end_note:
                return taan
        return None

    def decay_all(tg, factor=0.9):
        for (a, b), targets in tg.transitions.items():
            for c in targets:
                targets[c] *= factor

def bootstrap(tg, phrases: list[list[int]], weight=3.0):
    """
    Bootstraps the TaanGenerator with example phrases.
    
    Parameters:
    - tg: your TaanGenerator instance
    - phrases: list of phrases (each phrase is a list of MIDI offsets)
    - weight: reinforcement strength for each trigram
    """
    for phrase in phrases:
        for i in range(len(phrase) - 2):
            a, b, c = phrase[i], phrase[i+1], phrase[i+2]
            tg.observe(a, b, c, w=weight)
    tg.normalize()

from_swaras = lambda swaras: [swara_to_note(s) for s in swaras]

yaman_seed_phrases = [
    # from_swaras(["S", "R", "G", "M"]),
    # from_swaras(["M", "P", "D", "N"]),
    # from_swaras(["N", "S'", "N", "D", "P"]),
    # from_swaras(["G", "R", "S"]),
    # from_swaras(["P", "M", "G"]),
    # from_swaras(["R", "G", "M", "P"]),
    # from_swaras(["D", "P", "M", "G"]),
    # from_swaras(["M", "G", "R", "S"]),
    # from_swaras(["_N", "_S", "R", "G"]),
    from_swaras("_N R G M G R G M P D M P M D N R' N D N R' G' R' N R' S' N D P M P N N D P M P G M G R _N R S".split()),
    from_swaras("S' N D N S' R' S' N D N S' R' S' N D N S' G' R' S' N' S' R' R' S' N D P M P G M G R _N R S".split())
]

def taan_to_midi(taan, path, bpm=BPM, base_note=BASE_MIDI_NOTE):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # tempo: beats per minute → microseconds per beat
    tempo = int(60_000_000 / bpm)
    track.append(MetaMessage('set_tempo', tempo=tempo, time=0))

    tick_per_beat = mid.ticks_per_beat
    tick = tick_per_beat // 2  # 2 notes per beat = 0.5 beat per note

    def add_silence(beats):
        return Message("note_off", note=0, velocity=0, time=int(tick_per_beat * beats))

    def add_taan_sequence(seq):
        for note in seq:
            pitch = base_note + note
            track.append(Message("note_on", note=pitch, velocity=100, time=0))
            track.append(Message("note_off", note=pitch, velocity=100, time=tick))

    # === beat structure: [12 silent beats] + taan + [12 silent beats] + taan ===
    track.append(add_silence(12))          # pad 12 beats
    add_taan_sequence(taan)                # play taan
    track.append(add_silence(12))          # pad 12 beats
    add_taan_sequence(taan)                # play taan again

    mid.save(path)


def render_midi_to_wav(midi_path, wav_path, sf2_path):
    subprocess.run([
        "fluidsynth",
        "-g", "2.0",       # 🔊 boost gain
        "-ni", sf2_path,
        midi_path,
        "-F", wav_path,
        "-r", "48000"      # 🧩 match taal.wav sample rate
    ])

def mix_audio(taal_path, taan_path, out_path):
    taal = AudioSegment.from_wav(taal_path)
    taan = AudioSegment.from_wav(taan_path)

    # match volume (boost taan if needed)
    taan = taan + 3  # 🔊 increase volume by 6dB

    # optional trim/pad to align lengths
    taan = taan[:len(taal)]

    combined = taal.overlay(taan)
    combined.export(out_path, format="wav")

def taan_to_midi(taan, path, bpm=BPM, base_note=BASE_MIDI_NOTE):
    # prepend 12 silent beats and double
    # padded_taan = ([None]*24 + taan)*2

    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    tempo = int(60_000_000 / bpm)
    track.append(MetaMessage('set_tempo', tempo=tempo, time=0))

    tick_per_beat = mid.ticks_per_beat
    tick = int(tick_per_beat / 2)

    for note in taan:
        if note is None:
            # silent: rest for one note duration
            track.append(Message("note_off", note=0, velocity=0, time=tick))
        else:
            pitch = base_note + note
            track.append(Message("note_on", note=pitch, velocity=100, time=0))
            track.append(Message("note_off", note=pitch, velocity=100, time=tick))

    mid.save(path)

def reinforce(tg, taan, delta=0.1):
    for i in range(len(taan) - 3):  # ✂️ exclude last triple
        a, b, c = taan[i], taan[i+1], taan[i+2]
        tg.observe(a, b, c, w=delta)

def print_transition_matrix(tg, note_names=note_names, min_prob=0.01):
    print("\n=== TRANSITION MATRIX ===\n")
    for (a, b), nexts in sorted(tg.transitions.items()):
        readable_a = note_names.get(a, str(a))
        readable_b = note_names.get(b, str(b))
        print(f"({readable_a}, {readable_b}) →")

        for c, prob in sorted(nexts.items(), key=lambda x: -x[1]):
            if prob < min_prob:
                continue  # skip low-prob tails
            readable_c = note_names.get(c, str(c))
            bar = "█" * int(prob * 20)
            print(f"   {readable_c:>2}: {prob:.2f} {bar}")
        print()



# === MAIN ===
def main():
    tg = TaanGenerator(notes)
    bootstrap(tg, yaman_seed_phrases, weight=3.0)
    print_transition_matrix(tg)

    for round in range(10):
        t1 = tg.generate(BEATS, 0, 0)
        t2 = tg.generate(BEATS, 0, 0)

        taan_combination = [None] * 24 + t1 + [None] * 24 + t2


        taan_combined = taan_to_midi(taan_combination, "taan_combined.mid")
        render_midi_to_wav("taan_combined.mid", "taan_combined.wav", soundfont_path)
        mix_audio(taal_path, "taan_combined.wav", "taan_mixed.wav")

        subprocess.run(["ffplay", "-nodisp", "-autoexit", "taan_mixed.wav"])
        # subprocess.run(["ffplay", "-nodisp", "-autoexit", "t2_mixed.wav"])
    
        vote = input("Which taan was better? [1/2]: ").strip()
        if vote == "1":
            reinforce(tg, t1, delta=0.01)
            # punish(tg, t2, delta=0.2)
        elif vote == "2":
            reinforce(tg, t2, delta=0.01)
            # punish(tg, t1, delta=0.2)
    
        tg.normalize()
        print_transition_matrix(tg)


if __name__ == "__main__":
    main()




=== TRANSITION MATRIX ===

(_N, R) →
    S: 0.61 ████████████
    G: 0.31 ██████

(R, _N) →
    R: 0.87 █████████████████
   _P: 0.01 
   _D: 0.01 
   _N: 0.01 
    S: 0.01 
    G: 0.01 
    M: 0.01 
    P: 0.01 
    D: 0.01 
    N: 0.01 
   S': 0.01 
   R': 0.01 
   G': 0.01 

(R, G) →
    M: 0.87 █████████████████
   _P: 0.01 
   _D: 0.01 
   _N: 0.01 
    S: 0.01 
    R: 0.01 
    G: 0.01 
    P: 0.01 
    D: 0.01 
    N: 0.01 
   S': 0.01 
   R': 0.01 
   G': 0.01 

(G, R) →
   _N: 0.61 ████████████
    G: 0.31 ██████

(G, M) →
    G: 0.70 █████████████
    P: 0.24 ████

(M, G) →
    R: 0.91 ██████████████████

(M, P) →
    G: 0.38 ███████
    M: 0.19 ███
    D: 0.19 ███
    N: 0.19 ███

(M, D) →
    N: 0.77 ███████████████
   _P: 0.02 
   _D: 0.02 
   _N: 0.02 
    S: 0.02 
    R: 0.02 
    G: 0.02 
    M: 0.02 
    P: 0.02 
    D: 0.02 
   S': 0.02 
   R': 0.02 
   G': 0.02 

(P, G) →
    M: 0.87 █████████████████
   _P: 0.01 
   _D: 0.01 
   _N: 0.01 
    S: 0.01 
    R: 0.01 


ffplay version 6.1.1-3ubuntu5 Copyright (c) 2003-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

KeyboardInterrupt: Interrupted by user

In [93]:
import os
import random
import subprocess
from collections import defaultdict

from mido import Message, MidiFile, MidiTrack, MetaMessage
from pydub import AudioSegment

# === CONFIG ===
BPM = 200
BEATS = 12
BASE_MIDI_NOTE = 61

soundfont_path = os.path.expanduser("~/soundfonts/FluidR3_GM.sf2")
taal_path = os.path.expanduser("~/Documents/music/ektaal_200bpm_csharp.wav")

# === SWARA MAPPING ===
SWARA_DEGREES = {
    "S": 0, "R": 2, "G": 4, "M": 6, "P": 7, "D": 9, "N": 11,
}

def swara_to_note(swara: str) -> int:
    up = swara.count("'")
    down = swara.count("_")
    base = swara.strip("'_")
    if base not in SWARA_DEGREES:
        raise ValueError(f"Invalid swara: {swara}")
    return SWARA_DEGREES[base] + 12 * (up - down)

def note_to_swara(note: int) -> str:
    octave_offset, base_pitch = divmod(note, 12)
    inv = {v: k for k, v in SWARA_DEGREES.items()}
    if base_pitch not in inv:
        raise ValueError(f"Note {note} not mapped to swara")
    base = inv[base_pitch]
    if octave_offset == 0:
        return base
    elif octave_offset > 0:
        return base + ("'" * octave_offset)
    else:
        return ("_" * abs(octave_offset)) + base

notes = [-5, -3, -1, 0, 2, 4, 6, 7, 9, 11, 12, 14, 16]
note_names = {n: note_to_swara(n) for n in notes}
from_swaras = lambda swaras: [swara_to_note(s) for s in swaras]

# === MARKOV GENERATOR ===
class TaanGenerator:
    def __init__(self, notes, order=2):
        self.notes = notes
        self.order = order
        self.transitions = defaultdict(self._init_uniform)

    def _init_uniform(self):
        p = 1 / len(self.notes)
        return defaultdict(lambda: p, {n: p for n in self.notes})

    def observe(self, *history, next_note, w=1.0):
        if len(history) != self.order:
            raise ValueError(f"Expected {self.order} history elements")
        self.transitions[tuple(history)][next_note] += w

    def sample_next(self, history):
        history = tuple(history[-self.order:])
        options = self.transitions.get(history)
        if not options:
            return random.choice(self.notes)
        return random.choices(list(options), weights=list(options.values()))[0]

    def generate(self, beats, start_notes=None, end_note=None, tries=1000):
        if start_notes is None:
            start_notes = random.choices(self.notes, k=self.order)
        elif len(start_notes) != self.order:
            raise ValueError(f"start_notes must have {self.order} elements")
    
        for _ in range(tries):
            taan = list(start_notes)
            while len(taan) < beats * 2:
                next_note = self.sample_next(taan)
                taan.append(next_note)
            if end_note is None or taan[-1] == end_note:
                return taan
    
        return None  # failed to generate a valid taan


    def normalize(self):
        for key, targets in self.transitions.items():
            total = sum(targets.values())
            if total == 0:
                continue
            for k in targets:
                targets[k] /= total

    def decay_all(self, factor=0.99):
        for targets in self.transitions.values():
            for k in targets:
                targets[k] *= factor

# === DATASET BOOTSTRAPPING ===
def bootstrap(tg, phrases, weight=3.0):
    for phrase in phrases:
        for i in range(len(phrase) - tg.order):
            hist = tuple(phrase[i:i + tg.order])
            next_note = phrase[i + tg.order]
            tg.observe(*hist, next_note=next_note, w=weight)
    tg.normalize()

# === MIDI & AUDIO PIPELINE ===
def sequence_with_silence(taans, silence_beats, bpm=BPM):
    tick_per_note = 2  # 2 notes per beat
    silent = [None] * (silence_beats * tick_per_note)
    padded = []
    for taan in taans:
        padded += silent + taan
    return padded

def taan_to_midi(taan, path, bpm=BPM, base_note=BASE_MIDI_NOTE):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    tempo = int(60_000_000 / bpm)
    track.append(MetaMessage("set_tempo", tempo=tempo, time=0))

    tick_per_beat = mid.ticks_per_beat
    tick = tick_per_beat // 2

    for note in taan:
        if note is None:
            track.append(Message("note_off", note=0, velocity=0, time=tick))
        else:
            pitch = base_note + note
            track.append(Message("note_on", note=pitch, velocity=100, time=0))
            track.append(Message("note_off", note=pitch, velocity=100, time=tick))

    mid.save(path)

def render_midi_to_wav(midi_path, wav_path, sf2_path):
    subprocess.run([
        "fluidsynth", "-g", "2.0", "-ni", sf2_path,
        midi_path, "-F", wav_path, "-r", "48000"
    ])

def mix_audio(taal_path, taan_path, out_path):
    taal = AudioSegment.from_wav(taal_path)
    taan = AudioSegment.from_wav(taan_path)
    taan = taan + 3  # volume boost
    taan = taan[:len(taal)]
    combined = taal.overlay(taan)
    combined.export(out_path, format="wav")

# === LEARNING LOOP ===
def reinforce(tg, taan, delta=0.05):
    for i in range(len(taan) - tg.order - 1):
        hist = tuple(taan[i:i + tg.order])
        next_note = taan[i + tg.order]
        tg.observe(*hist, next_note=next_note, w=delta)

def print_transition_matrix(tg, note_names=note_names, min_prob=0.01):
    print("\n=== TRANSITION MATRIX ===\n")
    for hist, nexts in sorted(tg.transitions.items()):
        hist_str = "(" + ", ".join(note_names.get(n, str(n)) for n in hist) + ") →"
        print(hist_str)
        for c, p in sorted(nexts.items(), key=lambda x: -x[1]):
            if p < min_prob:
                continue
            print(f"   {note_names.get(c, str(c)):>3}: {p:.2f} {'█'*int(p*20)}")
        print()

# === EXAMPLE PHRASES ===
yaman_seed_phrases = [
    from_swaras("_N R G M G R G M P D M P M D N R' N D N R' G' R' N R' S' N D P M P N N D P M P G M G R _N R S".split()),
    from_swaras("S' N D N S' R' S' N D N S' R' S' N D N S' G' R' S' N' S' R' R' S' N D P M P G M G R _N R S".split())
]

# === MAIN ===
def main():
    tg = TaanGenerator(notes, order=4)
    bootstrap(tg, yaman_seed_phrases, weight=1.0)

    for round in range(10):
        start = [-1, 2, 4, 6]      # S R
        end = 0             # S
        
        t1 = tg.generate(BEATS, start_notes=start, end_note=end)
        t2 = tg.generate(BEATS, start_notes=start, end_note=end)

        combined_seq = sequence_with_silence([t1, t2], silence_beats=12)
        taan_to_midi(combined_seq, "taan_combined.mid")
        render_midi_to_wav("taan_combined.mid", "taan_combined.wav", soundfont_path)
        mix_audio(taal_path, "taan_combined.wav", "taan_mixed.wav")

        subprocess.run(["ffplay", "-nodisp", "-autoexit", "taan_mixed.wav"])

        vote = input("Which taan was better? [1/2]: ").strip()
        if vote == "1":
            reinforce(tg, t1)
        elif vote == "2":
            reinforce(tg, t2)

        tg.normalize()
        print_transition_matrix(tg)

if __name__ == "__main__":
    main()


FluidSynth runtime version 2.3.4
Copyright (C) 2000-2023 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'taan_combined.wav'..


ffplay version 6.1.1-3ubuntu5 Copyright (c) 2003-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

KeyboardInterrupt: Interrupted by user

In [94]:
def main():
    tg = TaanGenerator(notes, order=4)
    bootstrap(tg, yaman_seed_phrases, weight=1.0)

    while True:
        start = [-1, 2, 4, 6]  # _N R G M
        end = 0               # S

        taan = tg.generate(BEATS, start_notes=start, end_note=end)

        if taan is None:
            print("Generation failed. Try relaxing constraints.")
            continue

        combined_seq = sequence_with_silence([taan], silence_beats=12)
        taan_to_midi(combined_seq, "taan_single.mid")
        render_midi_to_wav("taan_single.mid", "taan_single.wav", soundfont_path)
        mix_audio(taal_path, "taan_single.wav", "taan_mixed.wav")

        subprocess.run(["ffplay", "-nodisp", "-autoexit", "taan_mixed.wav"])

        try:
            delta = float(input("Reinforce delta (positive to reward, negative to punish): ").strip())/2
        except ValueError:
            print("Invalid input. Skipping.")
            continue

        reinforce(tg, taan, delta)
        tg.normalize()
        print_transition_matrix(tg)

main()


FluidSynth runtime version 2.3.4
Copyright (C) 2000-2023 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'taan_single.wav'..


ffplay version 6.1.1-3ubuntu5 Copyright (c) 2003-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Reinforce delta (positive to reward, negative to punish):  -1



=== TRANSITION MATRIX ===

(_N, _N, S', G) →
    _P: 0.15 ███
    _D: 0.15 ███
    _N: 0.15 ███
     S: 0.15 ███
     R: 0.15 ███
     G: 0.15 ███
     M: 0.15 ███
     P: 0.15 ███
     D: 0.15 ███
     N: 0.15 ███
    S': 0.15 ███
    R': 0.15 ███

(_N, R, G, M) →
    _P: 0.08 █
    _D: 0.08 █
    _N: 0.08 █
     S: 0.08 █
     R: 0.08 █
     M: 0.08 █
     P: 0.08 █
     D: 0.08 █
     N: 0.08 █
    S': 0.08 █
    R': 0.08 █
    G': 0.08 █
     G: 0.08 █

(_N, S', G, G') →
    _P: 0.15 ███
    _D: 0.15 ███
     S: 0.15 ███
     R: 0.15 ███
     G: 0.15 ███
     M: 0.15 ███
     P: 0.15 ███
     D: 0.15 ███
     N: 0.15 ███
    S': 0.15 ███
    R': 0.15 ███
    G': 0.15 ███

(S, D, G, S') →
    _P: 0.15 ███
    _D: 0.15 ███
    _N: 0.15 ███
     S: 0.15 ███
     R: 0.15 ███
     G: 0.15 ███
     P: 0.15 ███
     D: 0.15 ███
     N: 0.15 ███
    S': 0.15 ███
    R': 0.15 ███
    G': 0.15 ███

(S, R', _N, _N) →
    _P: 0.15 ███
    _D: 0.15 ███
    _N: 0.15 ███
     S: 0.15 ███
     R:

ffplay version 6.1.1-3ubuntu5 Copyright (c) 2003-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

...

ffplay version 6.1.1-3ubuntu5 Copyright (c) 2003-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Reinforce delta (positive to reward, negative to punish):  -1



=== TRANSITION MATRIX ===

(_P, R, G', R') →
    _P: 0.15 ███
    _D: 0.15 ███
    _N: 0.15 ███
     S: 0.15 ███
     R: 0.15 ███
     G: 0.15 ███
     M: 0.15 ███
     P: 0.15 ███
     D: 0.15 ███
    S': 0.15 ███
    R': 0.15 ███
    G': 0.15 ███

(_P, G, D, _N) →
    _P: 0.15 ███
    _D: 0.15 ███
    _N: 0.15 ███
     S: 0.15 ███
     R: 0.15 ███
     G: 0.15 ███
     M: 0.15 ███
     P: 0.15 ███
     D: 0.15 ███
     N: 0.15 ███
    S': 0.15 ███
    R': 0.15 ███

(_P, P, _N, G') →
    _P: 0.08 █
    _D: 0.08 █
    _N: 0.08 █
     S: 0.08 █
     R: 0.08 █
     G: 0.08 █
     M: 0.08 █
     P: 0.08 █
     D: 0.08 █
     N: 0.08 █
    S': 0.08 █
    R': 0.08 █
    G': 0.08 █

(_D, _P, R, G') →
    _P: 0.15 ███
    _D: 0.15 ███
    _N: 0.15 ███
     S: 0.15 ███
     R: 0.15 ███
     G: 0.15 ███
     M: 0.15 ███
     P: 0.15 ███
     D: 0.15 ███
     N: 0.15 ███
    S': 0.15 ███
    G': 0.15 ███

(_N, _N, S', G) →
    _P: 0.15 ███
    _D: 0.15 ███
    _N: 0.15 ███
     S: 0.15 ███
    

ffplay version 6.1.1-3ubuntu5 Copyright (c) 2003-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

KeyboardInterrupt: Interrupted by user